## Random Forests

In [2]:
# Setup Libraries if needed
import pandas as pd
import sklearn
import numpy as np
import matplotlib as mpl
%matplotlib inline
mpl.pyplot.style.use('ggplot')

In [3]:
# import the training data
training = pd.read_csv('training.csv', index_col='idx')

# view the data
training.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30to59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60to89DaysPastDueNotWorse,NumberOfDependents
idx,,,,,,,,,,,
1,1,0.766127,45,2,0.802982,9120,13,0,6,0,2
2,0,0.957151,40,0,0.121876,2600,4,0,0,0,1
3,0,0.658180,38,1,0.085113,3042,2,1,0,0,0
4,0,0.233810,30,0,0.036050,3300,5,0,0,0,0
5,0,0.907239,49,1,0.024926,63588,7,0,1,0,0


In [4]:
# Import SciKit Learn functions
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import grid_search

In [5]:
# Split the target from the input variables
X = training.iloc[:, 1:]
y = training.iloc[:, 0]

# Now, split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [6]:
# Create Random Forest Object and fit
rf = RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=4, 
                            oob_score=True)
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=4,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [ ]:
# parameters = { 
#     'n_estimators': [200, 700,1000,1500,2000,5000],
#     'max_features': ['auto', 'log2'],
#     }
parameters = { 
    'n_estimators': [700,1000,1500,2000,3000],
    'max_features': ['auto', 'sqrt', 'log2'],
    }
# clftree = grid_search.GridSearchCV(RandomForestClassifier(criterion='gini', oob_score=True), parameters, n_jobs=4,
#                                   scoring='roc_auc',refit=True)
clftree = grid_search.RandomizedSearchCV(RandomForestClassifier(criterion='gini', oob_score=True), parameters, n_jobs=4,
                                  scoring='roc_auc',refit=True)
clftree.fit(X_train, y_train)
tree_model = clftree.best_estimator_
print (clftree.best_score_, clftree.best_params_) 

The Grid above was run on EC2 to save my poor computer from CPU overload. The results of the grid search  showed a negiible improvement on the AUC score. Nevertheless, we well train a model using the same specification on the entire training set and then predict the test set.

In [7]:
# Fit the optimal configuration to the training set and then 
# Look at the accuracy metrics

theForest = RandomForestClassifier(criterion='gini', oob_score=True ,n_estimators=2000,max_features='auto',
                                  n_jobs=-1) 

theForest.fit(X_train,y_train)

# Fit the decision tree classifier
theForest.predict(X_test)

tree_preds = theForest.predict_proba(X_test)[:, 1]

# Look at the accuracy metrics
print "The area under the curve is", roc_auc_score(y_test,tree_preds)
print theForest.oob_score_ 

The area under the curve is 0.849446067684


In [8]:
# Predict the test set
# import the test data
test = pd.read_csv('test.csv', index_col='idx')

X_ = test.iloc[:, 1:]

In [10]:
from datetime import datetime
startTime = datetime.now()
# Final training on entire set
theForest1 = RandomForestClassifier(criterion='gini', oob_score=True ,n_estimators=2000,max_features='auto',
                                  n_jobs=-1) 

theForest1.fit(X,y)

# Fit the decision tree classifier
theForest1.predict(X_)

tree_preds = theForest1.predict_proba(X_)[:, 1]

df = pd.DataFrame(tree_preds)

print datetime.now() - startTime

0:10:49.717000


In [11]:
df.to_csv('preds3.csv')

## Outcome of Public Score Submission: AUC = 0.852720